In [32]:
# импорт pandas выполняется один раз
import pandas as pd 

## XML

ИЗВЛЕКАЕМ КОНТЕНТ ИЗ XML-ФАЙЛА

Данные в формате XML имеют древовидную структуру. 

Что такое дерево? Это структура, которая имеет узлы и связи между ними. Самый верхнеуровневый узел называется корнем, а всё, что находится в самом низу, называется листьями. 

Для работы с XML-файлами мы будем использовать модуль ElementTree , входящий в стандартный пакет xml. Этот модуль позволит нам «перемещаться» по дереву XML и смотреть, что находится в каждом его узле, начиная с корня и заканчивая листьями.

Импортируем этот модуль под псевдонимом ET: 

In [33]:
# Импортируем модуль ElementTree
import xml.etree.ElementTree as ET

Для работы со структурой файла menu.xml считаем его содержимое в переменную tree, выполнив код:

In [34]:
tree = ET.parse('data/menu.xml')

Запишем в переменную root корневой узел дерева tree и посмотрим, как выглядит содержимое переменной root, для чего выполним код:

In [35]:
root = tree.getroot()
display(root)

<Element 'menu' at 0x00000295DBF1AB60>

Какой тип у этого объекта? Если мы вызовем встроенный в Python метод type() и передадим ему root , то увидим, что это тип xml.etree.ElementTree.Element. Такой тип будет у любого узла в дереве.

In [36]:
display(type(root))

xml.etree.ElementTree.Element

Для того чтобы посмотреть список потомков корневого узла, выполним следующий код:

In [37]:
display(list(root))

[<Element 'dish' at 0x00000295DBF1ACF0>,
 <Element 'dish' at 0x00000295DBF1B560>]

Если у узла нет потомков, то вернётся пустой список — [].

Для того чтобы получить список потомков второго блюда в нашем меню и вывести его на экран, выполним код:

In [38]:
display(list(root[1]))

[<Element 'price' at 0x00000295DBF1B6F0>,
 <Element 'weight' at 0x00000295DBF1A110>,
 <Element 'class' at 0x00000295DBF1A9D0>]

Таким образом, у второго потомка узла root —  три потомка.

Как было сказано ранее, у узлов могут быть параметры, или атрибуты. Например, у узлов dish есть атрибут name, который хранит название блюда.

Мы можем непосредственно обратиться к атрибутам, используя attrib.

Выведем на экран атрибуты первого блюда из меню:

In [39]:
display(root[0].attrib)

{'name': 'Кура'}

В XML-узлах часто хранятся количественные показатели. Эти показатели хранятся в виде текста, и прочитать их можно, обратившись к атрибуту text у соответствующего объекта типа ElementTree.Element.

Например, возьмём узел price первого блюда из меню:

In [40]:
display(root[0][0])

<Element 'price' at 0x00000295DBF1AAC0>

Теперь прочитаем значение этого узла с помощью text:

In [41]:
display(root[0][0].text)

'40'

Все значения в XML, даже числовые, хранятся как строки, поэтому преобразовывать их к нужному типу вам нужно самим.

Например, в данном случае можно обернуть значение стоимости в int() или float().

Если вы хотите прочитать наименование тега конкретного узла, необходимо использовать tag. Например, получим наименование тега корневого узла:

In [42]:
display(root.tag)
# Какое наименование имеет тег узла root[0][2]?
display(root[0][2].tag)

'menu'

'class'

На этом шаге мы решим задачу вывода на экран наименование всех блюд из меню, а также информацию о них (иными словами, нам необходимо обойти дерево и вывести на экран значения его листьев).

Используя цикл for, автоматизируем обход дерева. Для этого напишем следующий код:

In [43]:
for dish in root:
    for param in dish:
        print(dish.attrib['name'], param.tag, param.text)
    print()

Кура price 40
Кура weight 300
Кура class Мясо

Греча price 20
Греча weight 200
Греча class Крупа



✍️ В этом коде реализован следующий алгоритм:

В первом (внешнем) цикле перебираем потомков корня дерева (root). Потомки перебираются последовательно при помощи переменной dish. Это отдельные блюда из меню.
Во втором (вложенном) цикле аналогичным образом перебираем потомков каждого блюда. Этими потомками являются параметры блюда — его цена (price), вес (weight) и класс (class).
После этого выводим на экран название блюда (значение атрибута name), название очередного параметра (tag) и его значение (text).
Дополнительная функция print() в цикле верхнего уровня предназначена для организации более удобного восприятия информации — между отдельными блюдами будет выведена пустая строка.

#### XML. Загружаем, создаем, сохраняем

✍ Реализуем следующий алгоритм:

1) Загрузить данные из XML-файла menu.xml в переменную root.
2) Создать пустой список df_list (в него будем добавлять строчки итоговой таблицы).
3) Заранее создать список column_names с именами столбцов — название блюда (name), его цена (price), вес (weight) и класс (class).
4) В цикле организовать обход xml-дерева из корня по всем потомкам.
5) На каждой итерации цикла сформировать в виде списка строку таблицы, содержащую информацию: наименование блюда (атрибут name узла dish) и значения потомков этого узла — узлов price, weight, class.
6) Добавить сформированную строку в список df_list, используя метод append().
7) Сформировать из вложенного списка DataFrame. Имена для столбцов взять из списка column_names.

In [45]:
# Код, который реализует этот алгоритм:
import xml.etree.ElementTree as ET
tree = ET.parse('data/menu.xml')
root = tree.getroot()

import pandas as pd
column_names = ['name', 'price', 'weight', 'class']
df_list = []

for dish in root:
    row = [dish.attrib['name'], dish[0].text, dish[1].text, dish[2].text]
    df_list.append(row)

df = pd.DataFrame(df_list, columns=column_names)
display(df)

,name,price,weight,class
0,Кура,40,300,Мясо
1,Греча,20,200,Крупа


#### СОЗДАЁМ XML-ФАЙЛ

Чтобы создать корень дерева, используем метод Element() из класса ElementTree:

In [46]:
import xml.etree.ElementTree as ET

new_root = ET.Element('menu')
display(new_root)

<Element 'menu' at 0x00000295DBF1A480>

Теперь мы можем добавлять новые узлы в наше дерево, используя метод SubElement() из того же класса.

Добавим в наше меню двух потомков корневого узла, которые будут представлять два блюда, то есть будут узлами dish:

In [47]:
dish1 = ET.SubElement(new_root, 'dish', name='Кура')

dish2 = ET.SubElement(new_root, 'dish', name='Греча')

display(list(new_root))

[<Element 'dish' at 0x00000295DBFB3060>,
 <Element 'dish' at 0x00000295DBFB3100>]

В метод SubElement() мы передали первым аргументом узел, к которому добавляем потомка, вторым аргументом — наименование нового тега (dish),  третьим аргументом — наименование атрибута нового узла( name ) и его значение.

Добавим в создаваемую структуру по три потомка (атрибута) к двум новым узлам, которые будут содержать информацию о блюде — о его цене (price), весе (weight) и классе (class), а также значение этих атрибутов:

In [48]:
price1 = ET.SubElement(dish1, "price").text = "40"
weight1 = ET.SubElement(dish1, "weight").text = "300"
class1 = ET.SubElement(dish1, "class").text = "Мясо"
display(list(dish1))

price2 = ET.SubElement(dish2, "price").text = "20"
weight2 = ET.SubElement(dish2, "weight").text = "200"
class2 = ET.SubElement(dish2, "class").text = "Крупа"
display(list(dish2))

[<Element 'price' at 0x00000295DBFB32E0>,
 <Element 'weight' at 0x00000295DBFB3290>,
 <Element 'class' at 0x00000295DBFB2DE0>]

[<Element 'price' at 0x00000295DBFB3380>,
 <Element 'weight' at 0x00000295DBFB31A0>,
 <Element 'class' at 0x00000295DBFB31F0>]

Проверим визуально корректность созданной нами структуры, выполнив фрагмент кода, разработанного ранее:

In [49]:
for dish in new_root:    
    for param in dish:
        print(dish.attrib['name'], param.tag, param.text)
    print()

Кура price 40
Кура weight 300
Кура class Мясо

Греча price 20
Греча weight 200
Греча class Крупа



Созданная нами структура полностью идентична структуре исходного XML-файла.

#### СОХРАНЕНИЕ XML-ФАЙЛА

Преобразуем созданный нами объект типа ElementTree.Element в строку c помощью метода tostring(), передав наше новое дерево как аргумент. Сохраним эту строку на диске, используя стандартные средства Python:

In [50]:
new_root_string = ET.tostring(new_root)

with open("new_menu.xml", "wb") as f:
    f.write(new_root_string)

Возможно, вы увидите проблему, связанную с кодировкой. Что делать в этом случае? Как вариант — записать файл, используя сам класс ElementTree() :

In [51]:
ET.ElementTree(new_root).write('new_menu_good.xml', encoding="utf-8")

Для этого мы передаём в класс ElementTree() наше дерево (не его строковое представление) и вызываем метод write(). В метод мы передаём путь к новому файлу и нужную нам кодировку.